In [7]:
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import settings
import time

def db_connect():
    """
    Performs database connection using database settings from settings.py.
    Returns sqlalchemy engine instance
    """
    return create_engine(URL(**settings.DATABASE))

db = db_connect()

In [2]:
#get list of all tables in db
from sqlalchemy import MetaData
start_time = time.time()
m = MetaData(bind=db)
m.reflect()
tables = list(m.tables.keys())
time.time()-start_time

1.1059832572937012

In [3]:
# from list of tables, get all chartevent tables
import re
tab_re = re.compile(r'(chartevents.*)')
chart_tables = []
for table in tables:
    name=re.findall(tab_re, table)
    if name:
        chart_tables.append(name[0])
chart_tables = sorted(chart_tables)

In [6]:
#query itemid + distinct number of patients that itemid has been applied to
#save the query result in a csv file 
import time
#time.sleep(60*60) this was to give sam's query an hour to run
for chart in chart_tables:
    filename = chart + '.csv'
    start = time.time()
    que = '''select itemid, count(distinct subject_id) as num_patients
            from %s 
            group by itemid ''' % chart
    df = pd.read_sql(que, db)
    df.to_csv(filename)
    print(chart)
    print(time.time()-start)
    df = None

chartevents
3650.72926402092
chartevents_1
588.4106454849243
chartevents_10
103.94001913070679
chartevents_11
8.168391227722168
chartevents_12
3.973482847213745
chartevents_13
689.3983607292175
chartevents_14
605.5460398197174
chartevents_2
224.30061268806458
chartevents_3
875.8372004032135
chartevents_4
177.9776976108551
chartevents_5
372.21490001678467
chartevents_6
551.1637806892395
chartevents_7
4.786229610443115
chartevents_8
659.8748059272766
chartevents_9
1.976146936416626


In [46]:
#read all the csv files from the psql query, concat into a df
import glob
filenames=glob.glob('chartevents_*')
dflist=[]
for filename in filenames:
    df = pd.read_csv(filename, index_col = 0)
    dflist.append(df)

chart_df = pd.concat(dflist)

In [ ]:
#pull the d_items database (prepping to join to chart_df)
que = '''select * from d_items'''
d_items_df = pd.read_sql(que, db)

In [49]:
#joining the df
feat_df = chart_df.merge(d_items_df[['itemid', 'label']], on='itemid')
feat_df.sort_values(by='num_patients', ascending = False).head(50)

,itemid,num_patients,label
698,926,30762,Religion
2636,211,29998,Heart Rate
2672,31,29027,Activity
2718,80,26244,Bowel Sounds
5421,467,23574,O2 Delivery Device
692,916,22901,Allergy 1
2708,69,22901,BSA
701,935,22901,Allergy 3
700,930,22901,Ext
699,927,22901,Allergy 2


13